In [2]:
from pathlib import Path
import torch
import torch.nn as nn
from config import get_config, latest_weights_file_path
from train import get_model, get_ds, run_validation
from dataset import BilingualDataset


c:\Users\prabh\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
config = get_config()
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

# Load the pretrained weights
model_filename = latest_weights_file_path(config)
state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])

Using device: cuda
Max length of source sentence: 81
Max length of target sentence: 387


<All keys matched successfully>

In [4]:
run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: print(msg), 0, None, num_examples=10)

--------------------------------------------------------------------------------
    SOURCE:  write a python function to Check if a Substring is Present in a Given String

    TARGET: def checksubstring(string,sub_string):
	if(string.find(var_1)==-1):
	       return False
	else:
		return True
  
  
  

 PREDICTED: def check ( string , var_1 ) : 
      if ( string . find ( var_1 ) == - 1 ) : 
          print ( "NO" ) 
 else : 
          print ( "YES" ) 
 
 string = "geeks for geeks" 
 var_1 = "geek" 
 check ( string , var_1 ) 
 

--------------------------------------------------------------------------------
    SOURCE: 37. Python Program to Display Fibonacci Sequence Using Recursion

    TARGET: def recur_fibo(var_1):
   if var_1 <= 1:
       return var_1
   else:
       return(recur_fibo(var_1-1) + recur_fibo(var_1-2))

var_2 = 10


 PREDICTED: def recur_fibo ( n ) : 
      if n <= 1 : 
          return n 
 else : 
          return ( recur_fibo ( n - 1 ) + recur_fibo ( n - 2 ) ) 
 
 

In [60]:
from translate import translate
t = "write a python function to add calculate factorial of a number"

In [61]:
dp=[{'question':t,'solution':""}]
train_ds = BilingualDataset(dp, tokenizer_src, tokenizer_tgt, 'question', 'solution', config['seq_len'])

In [62]:
batch=train_ds[0]

In [63]:
from train import greedy_decode

In [64]:
encoder_input = batch["encoder_input"].unsqueeze(0).to(device) # (b, seq_len)
encoder_mask = batch["encoder_mask"].to(device) # (b, 1, 1, seq_len)

# check that the batch size is 1
assert encoder_input.size(
    0) == 1, "Batch size must be 1 for validation"

model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, 390, device)

source_text = batch["src_text"]
target_text = batch["tgt_text"]
model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())

# source_texts.append(source_text)
# expected.append(target_text)
# predicted.append(model_out_text)

# Print the source, target and model output
print(f"{f'SOURCE: ':>12}{source_text}")
print(f"{f'PREDICTED: ':>12}{model_out_text}")

    SOURCE: write a python function to add calculate factorial of a number
 PREDICTED: 
 def factorial ( n ) : 
      if n == 1 : 
          return 1 
 else : 
          return n * factorial ( n - 1 ) 
 

